# Super Resolution Generative Adversarial Network (SRGAN)



### Start with cloning repo
I used drive and local to get and save data. Tried GitHub bit it was too much trouble managing the commits.

In [ ]:
import os, shutil
from google.colab import files, drive

# save locations during training
drive.mount('/content/drive')

# unzip local file (preprocessed 176*176 images)
if not os.path.exists("coco"):
    !unzip coco
!rm coco.zip

# directories
data_dir = "coco/"                                        # input data directory
model_dir = "drive/My Drive/SRGAN_coco/model/"            # save model directory
output_dir = "drive/My Drive/SRGAN_coco/output/"          # save output directory

---
### Initialize (Hyper)parameters

With better GPU, higher batch sizes and image sizes could be used to improve the results.

In [ ]:
# =================
# (HYPER)PARAMETERS
# =================

# IMAGE
down_scale = 4                                       # downsampling scale
cropped_width = 176                                  # high res image (width)
cropped_height = 176                                 # high res image (height)
image_shape = (176, 176, 3)                          # high res image (shape)

# TRAIN
begin_epoch = 150                                    # the epoch to begin training at
num_images = 2000                                    # total number of images (train & test)
split_ratio = 0.8                                    # train-test split ratio
epochs = 1000                                        # number of epochs
batch_size = 32                                      # number of images for each batch
sample_every = 1                                     # number of epochs in between sampling
save_every = 25                                      # number of epochs in between saving model

---
### Preprocess Images

Crop center, down sample, then normalize. I chose to normalize both LR and HR images to [-1, 1].

**NOTE**:\
I ended up moving out most of preprocess functions out since in later training stages, I downloaded the outputs of these cells and zipped them for convenience. However, feel free to checkout the `utils.py` file for how I preprocessed the images and how I saved and loaded them.

In [ ]:
import numpy as np
from numpy import random
from PIL import Image
import matplotlib.pyplot as plt
import glob
%matplotlib inline

def down_sample(img, scale=down_scale):
    """Convert image to lower resolution."""
    new_h = img.shape[0]//scale;
    new_w = img.shape[1]//scale;
    lr_img = np.asarray(Image.fromarray(np.uint8(img)).resize((new_w, new_h), Image.BICUBIC))
    return lr_img

def normalize(img):
    """Normalize image to [-1,1]."""
    n_img = np.divide(img.astype(np.float32), 127.5) - np.ones_like(img, dtype=np.float32)
    return n_img

Collect preprocessed datasets.

In [ ]:
hr_images = []
lr_images = []

def get_data(data_dir=data_dir):
    """Populate 4D arrays of high res and low res images."""
    for img_name in glob.glob(data_dir + "*.jpg"):
        hr_img = np.asarray(Image.open(img_name))
        lr_img = down_sample(hr_img)
        hr_images.append(hr_img)
        lr_images.append(lr_img)
        
get_data()
print("High resolution image dataset shape:", np.shape(hr_images))
print("Low resolution image dataset shape:", np.shape(lr_images))

---
### Untility Function for Training

In [ ]:
def load_train_data(data_dir=data_dir, num_img=num_images, 
                    split_ratio=split_ratio, 
                    hr_images=hr_images, lr_images=lr_images):
    """Perform train-test split for high and low res images."""

    num_train = int(num_img * split_ratio)
    
    hr_images = [normalize(img) for img in hr_images]
    lr_images = [normalize(img) for img in lr_images]
    
    hr_train = hr_images[:num_train]
    hr_test = hr_images[num_train:]
    lr_train = lr_images[:num_train]
    lr_test = lr_images[num_train:]
    
    return hr_train, hr_test, lr_train, lr_test
    
# sanity check on dataset shapes
hr_train, hr_test, lr_train, lr_test = load_train_data()
print("HR image training dataset shape:", np.shape(hr_train), "\t")
print("LR image training dataset shape:", np.shape(lr_train), "\t")
print("HR image testing dataset shape:", np.shape(hr_test), "\t")
print("LR image testing dataset shape:", np.shape(lr_test), "\t")

---
### Build the SRGAN Neural Network

For visual details for on the network architecture, refer to the README.md file of the github repo.

In [ ]:
from keras.layers.core import Activation, Flatten
from keras.layers import Input, add, LeakyReLU, PReLU
from keras.layers import BatchNormalization, Conv2D, UpSampling2D, Dense
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
from keras.applications.vgg19 import VGG19
import keras.backend as K

def content_loss(y, y_pred):
    vgg19_model = VGG19(include_top=False, weights="imagenet", input_shape=image_shape)
    # Not trainable
    vgg19_model.trainable = False
    for layer in vgg19_model.layers:
        layer.trainable = False

    model = Model(inputs=vgg19_model.input, outputs=vgg19_model.get_layer("block5_conv4").output)
    model.trainable = False

    return K.mean(K.square(model(y) - model(y_pred)))

In [ ]:
from keras.models import load_model

d = load_model('coco_d_model{}.h5'.format(begin_epoch))
g = load_model('coco_g_model{}.h5'.format(begin_epoch), custom_objects={'content_loss': content_loss})

In [ ]:
def GAN_NN(g, d, shape, optimizer, content_loss):
    """Build and compile the GAN network to connect generator and discriminator."""
    d.trainable = False
    
    gan_input = Input(shape=shape)
    fake = g(gan_input)
    gan_output = d(fake)
    
    gan_model = Model(inputs=gan_input, outputs=[fake, gan_output])
    gan_model.compile(loss=[content_loss, "binary_crossentropy"], 
                loss_weights=[1., 1e-3], optimizer=optimizer)
    
    return gan_model

---
### Sample Generator Outputs

In [ ]:
def generate_image(e, g, hr_test, lr_test,
                    output_dir=output_dir,
                    dim=(1, 3), figsize=(15, 5)):
    """Util function for visualizing results of generator."""
    hr_batch = np.asarray(hr_test)
    lr_batch = np.asarray(lr_test)
    sr_batch = np.asarray(g.predict(lr_batch))
    
    # denormalize
    hr_batch = ((hr_batch + 1) * 127.5).astype(np.uint8)
    lr_batch = ((lr_batch + 1) * 127.5).astype(np.uint8)
    sr_batch = ((sr_batch + 1) * 127.5).astype(np.uint8)
    
    # random sample
    idx = random.randint(0, len(hr_test))

    # 1 row of 3 images (low res -> super res -> high res)
    plt.figure(figsize=figsize)
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(lr_batch[idx], interpolation="nearest")
    plt.axis("off")
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(sr_batch[idx], interpolation="nearest")
    plt.axis("off")
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(hr_batch[idx], interpolation="nearest")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(output_dir + "result_{}.png".format(e))
    plt.close()

---
### Set up File Structure to Store Training Results

In [ ]:
# tqdm gives more info about training
from tqdm import tqdm

def setup_training(data_dir=data_dir):
    """Make model and output directories, also initialize loss file."""
    if os.path.isdir(model_dir): shutil.rmtree(model_dir)
    if os.path.isdir(output_dir): shutil.rmtree(output_dir)
    if os.path.exists(model_dir + "loss.txt"): os.remove(model_dir + "loss.txt")
    
    os.mkdir(model_dir)
    os.mkdir(output_dir)
    loss_file = open(model_dir + "loss.txt" , "w+")
    loss_file.close()

setup_training()

---
### Training
For details on the training steps, refer to README.md.

In [ ]:
def SRGAN(epochs=epochs, batch_size=batch_size, split_ratio=split_ratio, 
          sample_every=sample_every, save_every=save_every,
          shape=image_shape, scale=down_scale, num_imgs=num_images,
          begin_epoch=begin_epoch, g=g, d=d):
    """The SRGAN training pipline."""
    
    hr_train, hr_test, lr_train, lr_test = load_train_data()
    
    num_batches = int(len(hr_train) // batch_size)
    shape_small = (shape[0]//scale, shape[1]//scale, shape[2])
    
    g_loss = content_loss
    gan = GAN_NN(g, d, shape_small, d.optimizer, g_loss)
    
    for e in range(begin_epoch+1, begin_epoch+epochs+1):
        for _ in tqdm(range(num_batches)):
            idxs = random.randint(0, len(hr_train), size=batch_size)
            hr_batch = []
            lr_batch = []
            hr_batch = [hr_train[i] for i in idxs]
            hr_batch = np.asarray(hr_batch)
            lr_batch = [lr_train[i] for i in idxs]
            lr_batch = np.asarray(lr_batch)
            sr_batch = g.predict(lr_batch)
            
            # std = 0.05, mean = 0.9 
            real_label = 0.05 * random.randn(batch_size) + 0.9
            # std = 0.05, mean = 0.1
            fake_label = 0.05 * random.randn(batch_size) + 0.1
            
            d.trainable = True
            d_loss_real = d.train_on_batch(hr_batch, real_label)
            d_loss_fake = d.train_on_batch(sr_batch, fake_label)
            d_loss = np.add(d_loss_real, d_loss_fake) / 2.0
            d.trainable = False

            idxs = random.randint(0, len(hr_train), size=batch_size)
            hr_batch = []
            lr_batch = []
            hr_batch = [hr_train[i] for i in idxs]
            hr_batch = np.asarray(hr_batch)
            lr_batch = [lr_train[i] for i in idxs]
            lr_batch = np.asarray(lr_batch)
            sr_batch = g.predict(lr_batch)
            
            # std = 0.05, mean = 0.9 
            gan_label = 0.05 * random.randn(batch_size) + 0.9
            
            gan_loss = gan.train_on_batch(lr_batch, [hr_batch, gan_label])
                    
        print("EPOCH {}\td_loss {}\tgan_loss {}".format(e, d_loss, gan_loss))

        loss_file = open(model_dir + "loss.txt" , "a")
        loss_file.write("EPOCH {}\td_loss {}\tgan_loss {}\n".format(e, d_loss, str(gan_loss)))
        loss_file.close()

        if (e == 1) or (e % sample_every == 0):
            generate_image(e, g, hr_test, lr_test)
        if (e % save_every == 0):
            g.save(model_dir + "coco_g_model{}.h5".format(e))
            d.save(model_dir + "coco_d_model{}.h5".format(e))

In [ ]:
# Train. Go to sleep and wake up tomorrow to find out that colab crashed and you lost all outputs :)
# deleted output since they are all logged in loss.txt file under model directory
SRGAN()

---
### View Some Results

Render result every 25 epochs. 

In [ ]:
# def visualize_result(output_dir=output_dir):
#     """View All Images."""
    
#     images_shown = []
#     for file_name in os.listdir(output_dir):
#         name, ext = file_name.split(".")
#         if int(name) / 25 == 0:
#             image = np.asarray(Image.open(file_name))
#             images_shown.append(image)

#     fig, ax = plt.subplots(1,len(images_shown))
#     for img in range(len(image_shown)):
#         ax[img].set_title("sample " + str(img+1))
#         ax[img].imshow(images_shown[img])